## Analysing moving average crossings

In [137]:
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from plotting import CandlePlot

In [141]:
# extend path to import modules from other folders
import sys
# append everything above the level where we're now
sys.path.append("../../")

In [142]:
# import instruments to calculate pips after trade signals have been identified
from code.infrastructure.instrument_collection import instrumentCollection as ic

ModuleNotFoundError: No module named 'code.infrastructure'; 'code' is not a package

In [ ]:
# loading data for an instruments pair
pair = "EUR_USD"
granularity = "H4"
df = pd.read_pickle(f"../../assets/data/{pair}_{granularity}.pkl")
MA_LIST = [10, 20]

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# get data frame with copy of relevant data
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [ ]:
# calculate the moving averages
# rolling() -> telling pandas to do rolling calculations
for ma in MA_LIST:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True) # inplace has to be set to True otherwise the dataframe won't get modified
# resetting the index
df_ma.reset_index(inplace=True, drop=True)

In [ ]:
df_ma.head()

In [ ]:
df_ma.info()

In [ ]:
# plotting the data frame with plotly
# take the first 500 candles of the data frame
df_plot = df_ma.iloc[:500]

In [ ]:
df_plot.shape

In [ ]:
# create candle plot instance
cp = CandlePlot(df_plot)

In [ ]:
traces = [ f"MA_{x}" for x in MA_LIST ]

In [ ]:
traces

In [ ]:
cp.show_plot(line_traces=traces)

## Deriving a strategy from average crossings

In [ ]:
MA_S = "MA_10"
MA_L = "MA_20"
BUY = 1
SELL = -1
NONE = 0

In [ ]:
# get data frame with copy of relevant data
df_an = df_ma[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', MA_S, MA_L]].copy()

In [ ]:
df_an.head()

In [ ]:
# calculate delta between short and long line
df_an['DELTA'] = df_an.MA_10 - df_an.MA_20

In [ ]:
df_an.head(25) # going from below to above the line at about row 14 -> this is a buy signal

In [ ]:
# to make a prediction when to buy we need to compare the delta with the DELTA of the day before
# therefore we add a new line where we shift the DELTA value by one day
df_an['DELTA_PREV'] = df_an.DELTA.shift(1)

In [ ]:
df_an.head(25) # going from below to above the line at about row 14 -> this is a buy signal

In [ ]:
# do we have a trade sign
def is_trade(row):
    # did we have a change and went above the line
    if row.DELTA >= 0 and row.DELTA_PREV < 0:
        return BUY
    elif row.DELTA < 0 and row.DELTA_PREV >= 0:
        return SELL
    return NONE

In [ ]:
# applying function and telling pandas to do it row by row (axis=1)
df_an['TRADE'] = df_an.apply(is_trade, axis=1)

In [ ]:
df_an.head(25)

In [ ]:
# get a dataframe with the interesting trades
df_trades = df_an[df_an.TRADE != NONE].copy()

In [ ]:
# how many trades do we have
df_trades.shape

In [ ]:
df_trades.head(5)

In [ ]:
# let's check if the crosses appear where they're supposed to appear
cp = CandlePlot(df_an.iloc[15:70])
cp.show_plot(line_traces=[MA_S, MA_L])

In [ ]:
# evaluate the trades in terms of Pips
# to gain insight how efective our trades were
# load instruments that have been imported at the top
ic.LoadInstruments("../../assets/data")

In [ ]:
ic

In [ ]:
ic.instruments_dict[pair] # loading the "EUR_USD" pair from the collection

In [ ]:
ins_data = ic.instruments_dict[pair]

In [ ]:
# accessing the gains that we've made
# looking at the data first
# example line 21 and 41:
# 21: we put a sell on 1.11386 (mid close)
# 41 then again we put a buy on 1.11398 (mid close)
df_trades.head()

In [ ]:
# highlight background color for mid_c row
# function taken from 
# https://stackoverflow.com/questions/44388149/colouring-one-column-of-pandas-dataframe
def highlight_col(x):
    r = 'background-color: green; color: white'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    
    df1.iloc[:, 4] = r
    return df1    

In [ ]:
# calculate with diff from pandas
# shift diffs up in minus 1 direction to make correct calculations
df_trades['DIFF'] = df_trades.mid_c.diff().shift(-1)
# filling Na's
df_trades.fillna(0, inplace=True)

In [ ]:
# line 21 to 41: price went up so difference is positive
# DIFF line 21 is mid_c line 41 minus mid_c line 21
df_trades.head().style.apply(highlight_col, axis=None)

In [ ]:
# convert diff into pips
df_trades['GAIN'] = df_trades['DIFF'] / ins_data.pipLocation
# needs to be multiplied to get correct gain or loss
df_trades['GAIN'] = df_trades['GAIN'] * df_trades['TRADE']

In [ ]:
df_trades.head().style.apply(highlight_col, axis=None)

In [ ]:
# calculate sum of the gains to see how it performed over time
# what's getting summarized here are pips
df_trades.GAIN.sum()

In [ ]:
# calculating cumulative gain
df_trades['GAIN_C'] = df_trades['GAIN'].cumsum()

In [ ]:
cp = CandlePlot(df_trades, candles=False)
cp.show_plot(line_traces=['GAIN_C'])
# plot shows a huge gain at the end of January 2020
# the time when the covid crises started and everything dropped except the US Dollar
# high pip value due to rise of US Dollar at this time
# doesn't tell much about how effective strategy is